Zhang Chao, 2025.04.07<br>
Plotting the provincial IWU trends

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader
from cartopy.feature import ShapelyFeature
from matplotlib.colors import Normalize
from matplotlib.colors import ListedColormap
import shapefile
import geopandas as gpd
import sys
import os
from scipy import stats
import matplotlib.lines as mlines
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')

df = pd.read_csv('IWU_Province.csv')

shp_CN     = shapereader.Reader('../shapefile/Province.shp')
shp_Nanhai = shapereader.Reader('../shapefile/Nanhai.shp')

# 颜色归一化
norm = Normalize(vmin=2500, vmax=15000)
n_cmap = plt.cm.ScalarMappable(norm=norm, cmap=plt.get_cmap('PuBu'))#gist_earth_r

color = ['r', 'b', 'gray']#'#C500FF'

In [3]:
def plot_series(fig,pos,df,pNo):
    ax = fig.add_axes(pos)
    plt.sca(ax)
    Y = df[df['PVC'] == pNo].iloc[0,1:]
    # print(PVA)
    pcolor = 0
    X = range(2003, 2021)
    # Y = PVA
    params = np.polyfit(X, Y, 1)
    a1 = params[0]
    b = params[1]
    r, sig = stats.spearmanr(X, Y)
    if sig < 0.06:
        if a1 > 0:
            pcolor = color[0]
        if a1 < 0:
            pcolor = color[1]
    else:
        pcolor = color[2]
    plt.plot(X, Y, c= pcolor, lw=2)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.axis('off')


def plot_shps(ax,shp_cn,shp_nh):
    ax.set_extent([80, 130,15,53], crs=ccrs.PlateCarree())
    for record, grid in zip(list(shp_cn.records()), list(shp_cn.geometries())):
        wu = record.attributes['m3_ha']
        facecolor = n_cmap.to_rgba(wu)
        ax.add_geometries([grid], ccrs.PlateCarree(),
                          edgecolor='black', facecolor=facecolor,
                          linewidth=0.5, alpha=0.7)
    
    shape_feature_NH = cfeature.ShapelyFeature(shp_nh.geometries(),
                                               ccrs.PlateCarree(), facecolor='1')
    ax.add_feature(shape_feature_NH, linewidth=0.5, edgecolor='black')
    
    return ax


def plot_nanhai(ax,shp_cn,shp_nh):
    extent = [106, 120, 4, 24]
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    shape_feature_CN = cfeature.ShapelyFeature(shp_cn.geometries(),
                                               ccrs.PlateCarree(), facecolor='1')
    ax.add_feature(shape_feature_CN, linewidth=0.5, edgecolor='black')
    
    shape_feature_NH = cfeature.ShapelyFeature(shp_nh.geometries(),
                                               ccrs.PlateCarree(), facecolor='1')
    ax.add_feature(shape_feature_NH, linewidth=0.5, edgecolor='black')


def plot_legend(ax,pos):
    # Create custom legend handles
    
    handles = [
        mlines.Line2D([],[],color='b',   label='slope<0 & p<0.05'),
        mlines.Line2D([],[],color='r',   label='slope>0 & p<0.05'),
        mlines.Line2D([],[],color='gray',label='slope=0  |  p>0.05'),
    ]
    
    ax.legend(handles=handles, ncol=1, loc='lower right',frameon=False,
              columnspacing = 3,fontsize=12,
              bbox_to_anchor=pos)


def add_colorbar(fig,pos,n_cmap):
    axc = fig.add_axes(pos)
    cb = plt.colorbar(n_cmap,cax=axc,orientation='horizontal')
    cb.ax.locator_params(nbins = 4)
    cb.ax.set_title('Mean water use [m$^{3}$/ha]',fontsize=12,transform=cb.ax.transAxes)
    
    
def plot_watertrend(fig,df):
    w, h = 0.08, 0.06
    # 华北
    plot_series(fig,[0.82, 0.58, w, h], df, 110000)  # 北京
    plot_series(fig,[0.82, 0.50, w, h], df, 120000)  # 天津
    plot_series(fig,[0.63, 0.57, w, h], df, 130000)  # 河北
    plot_series(fig,[0.60, 0.45, w, h], df, 410000)  # 河南
    plot_series(fig,[0.67, 0.52, w, h], df, 370000)  # 山东
    plot_series(fig,[0.68, 0.41, w, h], df, 340000)  # 安徽
    plot_series(fig,[0.64, 0.71, w, h], df, 150000)  # 内蒙古
    # # 西北区域
    plot_series(fig,[0.18, 0.64, w, h], df, 650000)  # 新疆
    plot_series(fig,[0.37, 0.58, w, h], df, 620000)  # 甘肃
    plot_series(fig,[0.34, 0.50, w, h], df, 630000)  # 青海
    plot_series(fig,[0.47, 0.52, w, h], df, 640000)  # 宁夏
    plot_series(fig,[0.52, 0.46, w, h], df, 610000)  # 陕西
    plot_series(fig,[0.57, 0.53, w, h], df, 140000)  # 山西
    # # #西南区域
    plot_series(fig,[0.23, 0.39, w, h], df, 540000)  # 西藏
    plot_series(fig,[0.43, 0.36, w, h], df, 510000)  # 四川
    plot_series(fig,[0.51, 0.36, w, h], df, 500000)  # 重庆
    plot_series(fig,[0.41, 0.22, w, h], df, 530000)  # 云南
    plot_series(fig,[0.50, 0.27, w, h], df, 520000)  # 贵州
    plot_series(fig,[0.54, 0.19, w, h], df, 450000)  # 广西
    # # 华南区域
    plot_series(fig,[0.55, 0.08, w, h], df, 460000)  # 海南
    plot_series(fig,[0.62, 0.21, w, h], df, 440000)  # 广东
    # # 华中
    plot_series(fig,[0.58, 0.30, w, h], df, 430000)  # 湖南
    plot_series(fig,[0.60, 0.38, w, h], df, 420000)  # 湖北
    # # #华东
    plot_series(fig,[0.72, 0.45, w, h], df, 320000)  # 江苏
    plot_series(fig,[0.82, 0.42, w, h], df, 310000)  # 上海
    # # 东南
    plot_series(fig,[0.66, 0.31, w, h], df, 360000)  # 江西
    plot_series(fig,[0.73, 0.36, w, h], df, 330000)  # 浙江
    plot_series(fig,[0.70, 0.27, w, h], df, 350000)  # 福建
    # # 东北
    plot_series(fig,[0.78, 0.81, w, h], df, 230000)  # 黑龙江
    plot_series(fig,[0.75, 0.71, w, h], df, 220000)  # 吉林
    plot_series(fig,[0.71, 0.64, w, h], df, 210000)  # 辽宁

In [4]:
plot_settings()
fig=plt.figure(figsize=(9,7.2))
lambert_proj = ccrs.LambertConformal(central_longitude=105, central_latitude=35, standard_parallels=(25, 47))
ax  = fig.add_axes([0.0,0.0,0.999,0.999],projection=lambert_proj,frame_on=False)
print(ax.get_position().width,ax.get_position().height)

plot_shps(ax,shp_CN,shp_Nanhai)


axn1 = fig.add_axes([0.88, 0.01, 0.1, 0.22], projection=lambert_proj)
plot_nanhai(axn1,shp_CN,shp_Nanhai)

ax.text(0.91, 0.60, 'Beijing',  fontsize=12,transform=ax.transAxes)  # 注记Beijing
ax.text(0.91, 0.52, 'Tianjin',  fontsize=12,transform=ax.transAxes)  # 注记Tianjin
ax.text(0.91, 0.44, 'Shanghai', fontsize=12,transform=ax.transAxes)  # 注记Shanghai

plot_watertrend(fig,df)

add_colorbar(fig,[0.04,0.23,0.25,0.03],n_cmap)

plot_legend(ax,[0.26,0.05])

plt.savefig('../figures/Figure_S01.png',dpi=300)

0.7992 0.999
